Note in all of these methods "c" must either be defined outside the function or added as a parameter. Preferably the latter honestly

In [ ]:
def ETDRK2(F,Total_Time, Number_of_Points, Initial_Condition, method = "taylor", debug = False):
    T = Total_Time
    N = int(Number_of_Points)
    u_0 = Initial_Condition

    h = (T/N)  ## calculate the step size based on the number of points
    tvec = np.arange(N+1)*h
    uvec = np.zeros(N+1)
    uvec[0] = u_0


    if debug:
        print("method is", method)
        print("h's value is", h, "type:", type(h))
    ## calculate coefficients to be used in the integral
    if method == "taylor":
        if debug:
            print("alpha")
        a1 = h   + (c*h**2)/2 + (c**2*h**3)/6 + (c**3*h**4)/24 ## (e^(ch)-1)/c
        b1 = h/2 + (c*h**2)/6 + (c**2*h**3)/24 ## (e^(ch) - (1 + ch))/(h*c^2)
        if debug:
            print("a1:", a1, ", b1:", b1)

    elif method == "contour":
        if debug:
            print("beta")
        M = 32  # number of points for complex means
        r = np.exp(1j * np.pi * (np.arange(1, M + 1)-0.5) / M)
        z = c*h + r
        a1 = h*np.mean((np.exp(z)-1)/z).real
        b1 =  h*np.mean((np.exp(z)-1-z)/(z**2)).real
        if debug:
            print("a1:", a1, ", b1:", b1)

    elif method == "naive":
        if debug:
            print("kappa")
        a1 = (np.exp(c*h)-1)/c
        b1 = (np.exp(c*h)-1-c*h)/(h*(c**2))
        if debug:
            print("a1:", a1, ", b1:", b1)
            
    else:
        raise ValueError("method must be contour, taylor or naive")




    for i in range(N):
        uvec[i+1] = uvec[i]*np.exp(c*h) + F(tvec[i])*a1 + (F(tvec[i+1])-F(tvec[i]))*b1
    
    return uvec,tvec

In [ ]:
def ETD2(Total_Time, Number_of_Points, Initial_Condition, method = "contour", debug = False):
    T = Total_Time
    N = Number_of_Points
    u_0 = Initial_Condition

    h = T/N
    tvec = np.arange(N+1)*h
    uvec = np.zeros(N+1)
    uvec[0] = u_0

    
    if method == "contour":
        z = c*h
        M = 32  # number of points for complex means
        r = np.exp(1j * np.pi * (np.arange(1, M + 1)-0.5) / M)
        z = c*h + r

        a1 = h*np.mean(((1+z)*np.exp(z) - 1 - 2*z)/(z**2))
        b1 = h*np.mean((1+z-np.exp(z))/(z**2))
        if debug:
            print("a1:", a1, ", b1:", b1)#
    
    elif method == "naive":
        a1 = ((1+c*h)*np.exp(c*h)-1-2*c*h)/(C**2*h)
        b1 = (1+c*h-np.exp(c*h))/(c**2*h)
        if debug:
            print("a1:", a1, ", b1:", b1)#
    else:
        raise ValueError("method must be contour or naive")

    return uvec,tvec

In [ ]:
def ETDRK4(Total_Time, Number_of_Points , Initial_Condition, method = "contour", debug = False):
    T = Total_Time
    N = int(Number_of_Points)
    u_0 = Initial_Condition
    h = T/N
    ## initialise my coefficients

    tvec = np.arange(N+1)*h
    uvec = np.zeros(N+1)
    uvec[0] = u_0

    if debug:
        print("h", h, type(h), "N", N)

    if method == "contour":
        M = 32  # number of points for complex means
        r = np.exp(1j * np.pi * (np.arange(1, M + 1)-0.5) / M)
        z = c*h + r

        a1 = h*np.mean((-4 -z +np.exp(z)*(4-3*z+z**2))/(z**3)).real
        b1 = h*np.mean((2 + z + np.exp(z)*(-2 +z))/(z**3)).real
        c1 = h*np.mean((-4-3*z -z**2 +np.exp(z)*(4-z))/(z**3)).real
    if method == "taylor":
        z = c*h

        a1 = (z/6 + z**3/24 + z**2/6 + z**4/24)/c
        b1 = (z/6 + z**2/12 + z**3/24)/c
        c1 = (z/6-z**3/24)/c
    if method == "naive":

        a1 = (-4 -h*c +np.exp(c*h)*(4 -3*h*c +(h*c)**2))/((h**2)*(c**3))
        b1 = (2 +h*c +np.exp(c*h)*(-2 +h*c))/((h**2)*(c**3))
        c1 = (-4 -3*h*c -(h*c)**2 + np.exp(c*h)*(4 -h*c))/((h**2)*(c**3))
    else:
        raise ValueError("method must be contour, taylor or naive")

    if debug:
        print("a1, b1, c1", a1, b1, c1, "method", method)


    for i in range(N):
        I = I_RK4(tvec[i],h,a1,b1,c1)
        uvec[i+1] = timestep(uvec[i], I, h)
    return uvec,tvec